In [40]:
import torch
import torch.nn as nn
import torch.optim as optim



In [13]:
data_test=  []
lab_test = []
# ones = True
# for i in range(10):
#     tdat  = []
#     tlab = []
    
#     for i in range(90):
#         tdat.append(i)
#         if ones:
#             tlab.append(1)
#         else:
#             tlab.append(0)
#     ones = not ones
#     data_test.append(tdat)
#     lab_test.append(tlab)
# X_train = torch.FloatTensor(data_test)
# X_test = X_train
# y_train = torch.FloatTensor(lab_test)
# y_test = y_train
import numpy as np

# Create an array of length 10, with each element being an array of length 90
data = np.random.rand(10, 90)

# Create labels for each element (assume binary labels)
labels = np.random.randint(0, 2, size=10)

# Split the data into X_train and y_train
X_train = data
y_train = labels
print(X_train)
print(y_train)
# Convert X_train and y_train to PyTorch tensors (if using PyTorch)
X_train_tensor = torch.from_numpy(X_train).float()
y_train_tensor = torch.from_numpy(y_train).float().unsqueeze(1)
X_train = X_train_tensor
y_train = y_train_tensor


[[3.50232660e-01 2.32136699e-01 8.11277462e-01 3.01458942e-01
  8.43743704e-01 6.18552738e-01 4.84170742e-01 4.49247982e-01
  6.01107706e-01 8.00476127e-01 8.37592257e-01 4.10103857e-02
  4.71232900e-01 5.39459367e-01 9.88971222e-01 9.90063197e-02
  7.69184319e-01 9.89446868e-01 5.08920067e-01 9.15425082e-01
  9.49061612e-01 6.84894299e-01 1.36353323e-01 7.19617475e-01
  4.60138286e-01 5.88353333e-01 2.89788817e-01 4.97804320e-01
  9.41821768e-01 6.22645170e-02 2.45885689e-01 6.50690207e-01
  2.88559059e-01 5.93098701e-01 4.74328829e-02 3.60385875e-02
  4.89803183e-01 3.78201609e-01 2.99956393e-01 4.12822095e-02
  9.19369697e-01 5.95085953e-01 4.38344896e-01 5.18198621e-01
  9.09117693e-01 2.53563217e-01 7.38408865e-01 8.70624249e-03
  3.56450734e-01 2.24388991e-01 1.59928614e-01 7.89323983e-01
  8.53541267e-01 2.21813552e-01 5.38695706e-01 2.59633579e-01
  8.55378366e-02 3.28484167e-01 1.55498724e-01 4.39271264e-01
  8.68373492e-01 9.66685298e-01 3.68114250e-01 5.13572774e-01
  4.3810

In [101]:
import os 
import glob
import pandas as pd
from torch.utils.data import Dataset
import numpy as np
import torch
from glob import glob

class TimeSeriesDataset(Dataset):
    def __init__(self, folder_nolayoff, folder_layoff, src_col):
        
        # csvs_layoffs = glob.glob(folder_layoff + "/*.csv")
        # csvs_nolayoffs = glob.glob(folder_nolayoff + "/*.csv")
        X_train = []
        Y_train = []
        all_csv_files = []
        EXT = "*.csv"  # Define the variable EXT
        for path, subdir, files in os.walk(folder_layoff):
            
            for file in glob(os.path.join(path, EXT)):
                # print(file)
                df = pd.read_csv(file, index_col=0, parse_dates=True).sort_index()
                ## If length of dataset is over 90, then cut it to 90
                if len(df) > 90:
                    print("OVER LENGTH", file)
                    df = df[-90:]
                ## If length of dataset is less than 90, then skip it
                if len(df) < 90:
                    continue
                X_train.append(df[src_col].values)
                Y_train.append(float(1))
                # all_csv_files.append(file)

        for path, subdir, files in os.walk(folder_nolayoff):
            for file in glob(os.path.join(path, EXT)):
                df = pd.read_csv(file, index_col=0, parse_dates=True).sort_index()
                ## If length of dataset is over 90, then cut it to 90
                if len(df) > 90:
                    print("OVER LENGTH", file)
                    df = df[-90:]

                ## If length of dataset is less than 90, then skip it
                if len(df) < 90:
                    continue
                X_train.append(df[src_col].values)
                Y_train.append(float(0))
                # all_csv_files.append(file)





        # for directory in csvs_layoffs:
        #     for csv in os.listdir(directory):
        #         if csv.endswith(".csv"):
        #             df = pd.read_csv(csv, index_col=0, parse_dates=True).sort_index()
        #             X_train.append(df[src_col].values)
        #             Y_train.append(1)
        #     # df = pd.read_csv(csv, index_col=0, parse_dates=True).sort_index()
        #     # X_train.append(df[src_col].values)
        #     # Y_train.append(1)
        # for directory in csvs_nolayoffs:
        #     for csv in os.listdir(directory):
        #         if csv.endswith(".csv"):
        #             df = pd.read_csv(csv, index_col=0, parse_dates=True).sort_index()
        #             X_train.append(df[src_col].values)
        #             Y_train.append(0)
            
        X_train = torch.from_numpy(np.array(X_train))
        Y_train = torch.from_numpy(np.array(Y_train))
        # if len(X_train.shape) < 3:
        #     X_train = X_train.unsqueeze(2)

        # if X_train.shape.index(min(X_train.shape[1], X_train.shape[2])) != 1:  # make sure the Channels in second dim
        #     X_train = X_train.permute(0, 2, 1)

        self.X_train = X_train
        self.Y_train = Y_train
        self.num_channels = X_train.shape[1]
        self.len = X_train.shape[0]

    def __len__(self):
        return self.len
    
    def __getitem__(self, idx):
        return self.X_train[idx].float(), self.Y_train[idx]



In [33]:
from torch.utils.data import DataLoader

In [102]:
tdloader = TimeSeriesDataset("stocks_no_layoffs", "stocks_layoffs", "open_percent_change")

C:\Users\alexs\AppData\Local\Temp\ipykernel_4548\1456045152.py:22: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df = pd.read_csv(file, index_col=0, parse_dates=True).sort_index()
C:\Users\alexs\AppData\Local\Temp\ipykernel_4548\1456045152.py:22: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df = pd.read_csv(file, index_col=0, parse_dates=True).sort_index()
C:\Users\alexs\AppData\Local\Temp\ipykernel_4548\1456045152.py:22: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df = pd.read_csv(file, index_col=0, parse_dates=True).sort_index()
C:\Users\alexs\AppData\Local\Temp\ipyker

OVER LENGTH stocks_layoffs\Data\MTTR1.csv
OVER LENGTH stocks_layoffs\Food\TOST1.csv


C:\Users\alexs\AppData\Local\Temp\ipykernel_4548\1456045152.py:22: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df = pd.read_csv(file, index_col=0, parse_dates=True).sort_index()
C:\Users\alexs\AppData\Local\Temp\ipykernel_4548\1456045152.py:22: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df = pd.read_csv(file, index_col=0, parse_dates=True).sort_index()
C:\Users\alexs\AppData\Local\Temp\ipykernel_4548\1456045152.py:22: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df = pd.read_csv(file, index_col=0, parse_dates=True).sort_index()
C:\Users\alexs\AppData\Local\Temp\ipyker

OVER LENGTH stocks_layoffs\Real Estate\COMP1.csv
OVER LENGTH stocks_layoffs\Real Estate\COMP2.csv
OVER LENGTH stocks_layoffs\Real Estate\OPEN1.csv
OVER LENGTH stocks_layoffs\Real Estate\OPEN2.csv
OVER LENGTH stocks_layoffs\Recruiting\ZIP1.csv


C:\Users\alexs\AppData\Local\Temp\ipykernel_4548\1456045152.py:22: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df = pd.read_csv(file, index_col=0, parse_dates=True).sort_index()
C:\Users\alexs\AppData\Local\Temp\ipykernel_4548\1456045152.py:22: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df = pd.read_csv(file, index_col=0, parse_dates=True).sort_index()
C:\Users\alexs\AppData\Local\Temp\ipykernel_4548\1456045152.py:22: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df = pd.read_csv(file, index_col=0, parse_dates=True).sort_index()
C:\Users\alexs\AppData\Local\Temp\ipyker

...

C:\Users\alexs\AppData\Local\Temp\ipykernel_4548\1456045152.py:36: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df = pd.read_csv(file, index_col=0, parse_dates=True).sort_index()
C:\Users\alexs\AppData\Local\Temp\ipykernel_4548\1456045152.py:36: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df = pd.read_csv(file, index_col=0, parse_dates=True).sort_index()
C:\Users\alexs\AppData\Local\Temp\ipykernel_4548\1456045152.py:36: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df = pd.read_csv(file, index_col=0, parse_dates=True).sort_index()
C:\Users\alexs\AppData\Local\Temp\ipyker

OVER LENGTH stocks_no_layoffs\Recruiting\ZIP1.csv
OVER LENGTH stocks_no_layoffs\Security\AURA0.csv


C:\Users\alexs\AppData\Local\Temp\ipykernel_4548\1456045152.py:36: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df = pd.read_csv(file, index_col=0, parse_dates=True).sort_index()
C:\Users\alexs\AppData\Local\Temp\ipykernel_4548\1456045152.py:36: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df = pd.read_csv(file, index_col=0, parse_dates=True).sort_index()
C:\Users\alexs\AppData\Local\Temp\ipykernel_4548\1456045152.py:36: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df = pd.read_csv(file, index_col=0, parse_dates=True).sort_index()
C:\Users\alexs\AppData\Local\Temp\ipyker

OVER LENGTH stocks_no_layoffs\Travel\ISPO0.csv
OVER LENGTH stocks_no_layoffs\Travel\ISPO1.csv
OVER LENGTH stocks_no_layoffs\Travel\ISPO2.csv
OVER LENGTH stocks_no_layoffs\Travel\SOND1.csv
OVER LENGTH stocks_no_layoffs\Travel\SOND2.csv
OVER LENGTH stocks_no_layoffs\Travel\SOND3.csv


C:\Users\alexs\AppData\Local\Temp\ipykernel_4548\1456045152.py:36: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df = pd.read_csv(file, index_col=0, parse_dates=True).sort_index()
C:\Users\alexs\AppData\Local\Temp\ipykernel_4548\1456045152.py:36: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df = pd.read_csv(file, index_col=0, parse_dates=True).sort_index()
C:\Users\alexs\AppData\Local\Temp\ipykernel_4548\1456045152.py:36: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df = pd.read_csv(file, index_col=0, parse_dates=True).sort_index()


In [88]:
layoff_data_loader = DataLoader(dataset=tdloader, batch_size=1, 
                                shuffle=True, drop_last=False, num_workers=0)

In [89]:
layoff_data_loader.dataset.X_train.shape

torch.Size([551, 90])

In [90]:
## Iterate through dataloader and print y labels
for i, (x, y) in enumerate(layoff_data_loader):
    print(y)
    # print(x)
    # break

tensor([1.], dtype=torch.float64)
tensor([1.], dtype=torch.float64)
tensor([0.], dtype=torch.float64)
tensor([0.], dtype=torch.float64)
tensor([0.], dtype=torch.float64)
tensor([0.], dtype=torch.float64)
tensor([1.], dtype=torch.float64)
tensor([0.], dtype=torch.float64)
tensor([1.], dtype=torch.float64)
tensor([0.], dtype=torch.float64)
tensor([0.], dtype=torch.float64)
tensor([0.], dtype=torch.float64)
tensor([0.], dtype=torch.float64)
tensor([0.], dtype=torch.float64)
tensor([0.], dtype=torch.float64)
tensor([1.], dtype=torch.float64)
tensor([1.], dtype=torch.float64)
tensor([0.], dtype=torch.float64)
tensor([0.], dtype=torch.float64)
tensor([1.], dtype=torch.float64)
tensor([1.], dtype=torch.float64)
tensor([0.], dtype=torch.float64)
tensor([1.], dtype=torch.float64)
tensor([0.], dtype=torch.float64)
tensor([0.], dtype=torch.float64)
tensor([1.], dtype=torch.float64)
tensor([0.], dtype=torch.float64)
tensor([0.], dtype=torch.float64)
tensor([1.], dtype=torch.float64)
tensor([1.], d

In [100]:
# Define the model architecture
class BinaryClassifier(nn.Module):
    def __init__(self):
        super(BinaryClassifier, self).__init__()
        self.fc1 = nn.Linear(90, 64)
        self.fc2 = nn.Linear(64, 32)
        self.fc3 = nn.Linear(32, 1)
        self.relu = nn.ReLU()
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        x = self.sigmoid(self.fc3(x))
        return x

# Create an instance of the model
model = BinaryClassifier()

# Define the loss function and optimizer
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters())

# Training loop
num_epochs = 50
batch_size = 32

for epoch in range(num_epochs):
    ## Run through dataloader batches
    



    for i, (X_train, y_train) in enumerate(layoff_data_loader):
        print(X_train)
        print(y_train)
        # Forward pass
        outputs = model(X_train)
        # print(outputs.unsqueeze(0))
        print(outputs)
        # Convert float to binary:
        # print()
        # out = outputs.squeeze(1)
        # output_bin = (out >= 0.5).int()
        # print(output_bin)
        y_train = y_train.unsqueeze(1)
        loss = criterion(outputs.float(), y_train.float())

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
    # for i in range(0, len(X_train), batch_size):
    #     batch_X = X_train[i:i+batch_size]
    #     batch_y = y_train[i:i+batch_size]

    #     # Forward pass
    #     outputs = model(batch_X)
    #     loss = criterion(outputs, batch_y)

    #     # Backward pass and optimization
    #     optimizer.zero_grad()
    #     loss.backward()
    #     optimizer.step()

    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

# # Evaluation
# with torch.no_grad():
#     outputs = model(X_test)
#     predicted = (outputs >= 0.5).float()
#     accuracy = (predicted == y_test).float().mean()
#     print(f'Test Accuracy: {accuracy:.4f}')

# # Make predictions
# with torch.no_grad():
#     outputs = model(X_new)
#     predicted = (outputs >= 0.5).float()

tensor([[ 0.0000,  0.0000,  0.0000,  0.0000, -0.0018, -0.0018, -0.0018, -0.0018,
         -0.1235, -0.0113, -0.0765, -0.0299,  0.0137,  0.0136,  0.0134, -0.0116,
          0.0120,  0.0428,  0.0005, -0.0015, -0.0015, -0.0015, -0.0277, -0.0309,
          0.0403, -0.0258,  0.0156,  0.0154,  0.0152, -0.0224,  0.0160,  0.0522,
         -0.0235,  0.0118,  0.0117,  0.0115, -0.0608, -0.0305, -0.0381, -0.0296,
          0.0058,  0.0058,  0.0057, -0.0517, -0.0395,  0.0434, -0.1588,  0.0130,
          0.0128,  0.0127, -0.0241,  0.0021,  0.0112, -0.0063, -0.0047, -0.0047,
         -0.0048, -0.0029, -0.0340, -0.0135, -0.0098,  0.0060,  0.0060,  0.0059,
          0.0059, -0.0415,  0.0413,  0.0238, -0.0007, -0.0007, -0.0007,  0.0100,
          0.0476,  0.0233, -0.0196, -0.0075, -0.0076, -0.0076, -0.0517, -0.0125,
          0.0109,  0.0072,  0.0048,  0.0048,  0.0048, -0.0386,  0.0044,  0.0335,
          0.0298,  0.0000]])
tensor([0.], dtype=torch.float64)
tensor([[0.5269]], grad_fn=<SigmoidBackward0>)

RuntimeError: all elements of input should be between 0 and 1